In [1]:
import pandas as pd 
import torch
import numpy as np
from matplotlib import pyplot as plt
#from torchmetrics import JaccardIndex
import seaborn as sbn
from sklearn.metrics import roc_auc_score
import time
import torchvision
import torchvision.transforms as transforms

import sys 
sys.path.append('../src/')
from data_loading import load_tabular_data, preprocess_data, corrupt_label
from GenDataset import GenDataset
from DVGS import DVGS
from DVRL import DVRL
from utils import get_corruption_scores
from NN import NN
from CNN import CNN

%load_ext autoreload
%autoreload 2

/home/teddy/miniconda3/envs/dvgs/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
noise_idx = load_tabular_data('adult', {'train':1000, 'valid':400}, noise_rate=0.) # saves to disk
x_train, y_train, x_valid, y_valid, x_test, y_test, col_names = preprocess_data('minmax', 'train.csv', 'valid.csv', 'test.csv')
train_dataset = GenDataset(x_train, y_train)
test_dataset = GenDataset(x_test, y_test)
valid_dataset = GenDataset(x_valid, y_valid)

model = NN(in_channels=108, out_channels=1, num_layers=1, hidden_channels=100, norm=False, dropout=0., bias=True, act=torch.nn.ReLU, out_fn=torch.nn.Sigmoid())
dvgs = DVGS(train_dataset, valid_dataset, test_dataset, model)
#dvgs.pretrain_(crit=torch.nn.BCELoss(), num_workers=1, batch_size=256, lr=1e-3, epochs=25)
print()

tic = time.time() 
vals_dvgs = dvgs.run(crit          = torch.nn.BCELoss(), 
                    similarity     = torch.nn.CosineSimilarity(dim=0), 
                    optim          = torch.optim.Adam, 
                    lr             = 1e-2, 
                    num_epochs     = 100, 
                    compute_every  = 5, 
                    batch_size     = 256, 
                    num_workers    = 1, 
                    grad_params    = None, 
                    verbose        = True, 
                    use_cuda       = False)
print()
print(f'time elapsed: {(time.time() - tic)/60:.2f} min')